Winship Project, generate biomarkerId tag from label tag.

In [43]:
import girder_client

apiUrl = "http://winship.digitalslidearchive.emory.edu:8080/api/v1"
gc = girder_client.GirderClient(apiUrl=apiUrl)
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [51]:
collectionId = '5b88a29fe629140099a09c32'
import re
regexFormat = re.compile('^(?P<biobankId>EUH(-|_)\d{1,4})_*')

biobankIds = set()
# Get folders in collection.
for fld in gc.listFolder(collectionId):
    # Get the items in each folder. 
    for item in gc.listItem(fld['_id']):
        if 'meta' in item and 'label' in item['meta']:
            # biobankId is a subset of the label, regex to capture it.
            label = item['m eta']['label']
            m = regexFormat.search(label)
            if m:
                m = m.groupdict()
                m['biobankId'] = m['biobankId'].replace('_','-')
                gc.addMetadataToItem(item['_id'], m) # Push biobankId metadata.
                biobankIds.add(m['biobankId'])

In [54]:
import json
virtualFolderId = '5bc66055e629140099673c0d'
baseFolderId = '5bc66055e629140099673c0d'
 
for _id in biobankIds:
    params = { 
        "parentType":    "folder",
        "parentId":    virtualFolderId,
        "reuseExisting":     True, 
        "name":            _id,
        "description":     _id,
        "isVirtual":         True,
        "virtualItemsQuery": json.dumps({"meta.biobankId": _id, 
                                        'baseParentId': {"$oid": baseFolderId}})}
    gc.post("folder", parameters=params)